Begin to implement chains and implement a pipeline 

In [1]:
with open("job_desc.md") as f:
    description = f.read()
with open("resume.md") as f:
    resume = f.read()
title = 'django developer'

In [2]:
from util import chunk_markdown, cut_sections

In [3]:
sections = cut_sections(resume)

history = sections['history']
chunks = chunk_markdown(history)

We need a function to only select job history from chunks. Any other headings such as summary, or education should be set aside.

We need to split the job section into 2 parts, the job experience description and the job metadata.

## Begin to process job history chunks

Implementing `beef_section_chain()`

In [4]:
from beef import highlight_chain
from util import relevant_skills_chain

In [5]:
from util import job_requirement_chain


requirements = job_requirement_chain().predict(desc=description)


In [9]:
from langchain.chains import SequentialChain

beef_chain = SequentialChain(
    chains=[relevant_skills_chain(), highlight_chain()],
    input_variables=["title", "desc", "section", "requirements"],
    output_variables=["highlighted"]
)

for chunk in chunks:
    print(beef_chain({"section": chunk.page_content, "title": title, "desc": description, "requirements": requirements})['highlighted'])

- Results-driven:
  - Assisted primary supervisor in management of maintenance programs leading to no hits on a major inspection.
  - Expertly managed tool program of over $175,000 by training coworkers on expectations and proper procedures leading to no hits across 2 major and 3 routine inspections.
  - Awarded for showing expertise and initiative for reinstalling an incorrectly installed part under stressful conditions.
- Teamwork:
  - Lead and mentored a group of 3 individuals in flight deck operations and proper flight deck cleaning procedures as supervisor leading to no accidents and rejuvenating a stagnant work center.
- Fast Learner:
  - Responsible for maintaining electrical and instrument systems for 11 F-18E aircraft totaling $715 million.
  - Responsible for final check and troubleshooting of aircraft before flight.
- Developed automated trading system in Python using parallel processing, AWS, REST API's, and SQL, demonstrating proficiency in Python and software engineering.

It seems that a router should be used to evaluate a job experience section. If a job experience section is not strong for a given job description, THAN the chains described in this notebook should be used. This chain should be called `BeefSectionChain`. It's intention should be to highlight the key skills needed for a given job description.

When processing the job experience section as a whole, a comparison should be made between the processed section, and the original section. The section that returns `"more appealing to a recruiter"` should be used as `Document.page_content`.

How can we build an excellent resume summary from the job experience sections?

First, each job experience will be filtered with a prompt such as `"what is one thing that a recruiter for x job needs to know"`. Then that list of facts will be reduced to 3. Then the original will be updated with `"make this appealing for a recruiter for x job"`. The two proposed sections will be compared.

Or maybe model can just condense, then choose 3-5 of the best lines.